In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
from fastai import *
from fastai.vision import *

In [ ]:
# to get all files from a directory
import os

# to easier work with paths
from pathlib import Path

# to read and manipulate .csv-files
import pandas as pd

In [ ]:
INPUT = Path("../input/digit-recognizer")


In [ ]:
train_df = pd.read_csv(INPUT/"train.csv")
train_df.head(3)

In [ ]:
test_df = pd.read_csv(INPUT/"test.csv")
test_df.head(3)
test_df.info()

In [ ]:
TRAIN = Path("../train")
TEST = Path("../test")

In [ ]:
# Create training directory
for index in range(10):
    try:
        os.makedirs(TRAIN/str(index))
    except:
        pass

In [ ]:
#Create test directory
try:
    os.makedirs(TEST)
except:
    pass

In [ ]:
# import numpy to reshape array from flat (1x784) to square (28x28)
import numpy as np

# import PIL to display images and to create images from arrays
from PIL import Image

def saveDigit(digit, filepath):
    digit = digit.reshape(28,28)
    digit = digit.astype(np.uint8)

    img = Image.fromarray(digit)
    img.save(filepath)

In [ ]:
# save training images
for index, row in train_df.iterrows():
    
    label,digit = row[0], row[1:]
    
    folder = TRAIN/str(label)
    filename = f"{index}.jpg"
    filepath = folder/filename
    
    digit = digit.values
    
    saveDigit(digit, filepath)

In [ ]:
# save testing images
for index, digit in test_df.iterrows():

    folder = TEST
    filename = f"{index}.jpg"
    filepath = folder/filename
    
    digit = digit.values
    
    saveDigit(digit, filepath)

In [ ]:
subdirectory=str(0)
path = TEST
images = os.listdir(path)
image = Image.open(path/images[2])
image

In [ ]:
import matplotlib.pyplot as plt
image_path = TEST/os.listdir(TEST)[9]
image = Image.open(image_path)
image_array = np.asarray(image)


fig, ax = plt.subplots(figsize=(15, 15))

img = ax.imshow(image_array, cmap='gray')

for x in range(28):
    for y in range(28):
        value = round(image_array[y][x]/255.0, 2)
        color = 'black' if value > 0.5 else 'white'
        ax.annotate(text=value, xy=(x, y), ha='center', va='center', color=color)

plt.axis('off')
plt.show()

In [ ]:
data = ImageDataLoaders.from_folder(
    path = TRAIN,
    test = TEST,
    valid_pct = 0.2,
    bs = 16,
    size = 28,
    #num_workers = 0,
    
)

In [ ]:
learn = cnn_learner(data, resnet34, metrics=error_rate)
lr_min,lr_steep = learn.lr_find()

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learn = cnn_learner(data, resnet34, metrics=accuracy)


In [ ]:
learn.fit_one_cycle(3, 1e-4)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_confusion_matrix()


In [ ]:
path = TEST
images=os.listdir(TEST)
predictions=[]
image = Image.open(path/images[5])
image
for index in range(0,len(test_df)):
    p=learn.predict(path/images[index])
    class_score=p[0]
    predictions.append(class_score)


In [ ]:
len(predictions)

In [ ]:
predictions[1:20]

In [ ]:
sample_submission =  pd.read_csv(INPUT/"sample_submission.csv")
display(sample_submission.head(2))
display(sample_submission.tail(2))

In [ ]:
# remove file extension from filename
ImageId = [os.path.splitext(path)[0] for path in os.listdir(TEST)]
# typecast to int so that file can be sorted by ImageId
ImageId = [int(path) for path in ImageId]
# +1 because index starts at 1 in the submission file
ImageId = [ID+1 for ID in ImageId]

In [ ]:
submission  = pd.DataFrame({
    "ImageId": ImageId,
    "Label": predictions
})
# submission.sort_values(by=["ImageId"], inplace = True)
submission.to_csv("submission.csv", index=False)
display(submission.head(3))
display(submission.tail(3))

In [ ]:
learn.export??


In [ ]:
learn.export(fname = Path("/kaggle/working/export.pkl"))